In [57]:
# ! pip install mlxtend
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

sales_fact_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_1998.csv")
sales_fact_dec_1998_ori_df = pd.read_csv("./hw1_data/sales_fact_dec_1998.csv")
product_ori_df = pd.read_csv("./hw1_data/product.csv")
# display(sales_fact_1998_ori_df.info(), sales_fact_dec_1998_ori_df.info(), product_ori_df.info())

time: 261 ms (started: 2022-01-04 07:01:15 +00:00)


In [58]:
transaction_ori_df = pd.concat([sales_fact_1998_ori_df, sales_fact_dec_1998_ori_df]).reset_index()
transaction_df = transaction_ori_df.copy().drop(['promotion_id', 'unit_sales', 'index', 'store_sales', 'store_cost'], axis=1)
transaction_df['product_id'] = transaction_ori_df['product_id'].apply(lambda x:product_ori_df.loc[product_ori_df['product_id']==x, 'product_name'].values[0])
transaction_df = pd.concat([transaction_df, pd.get_dummies(transaction_df['product_id'])], axis=1).drop(['product_id'],axis=1)
transaction_df = transaction_df.groupby(['time_id', 'customer_id', 'store_id']).sum()

time: 57.3 s (started: 2022-01-04 07:01:15 +00:00)


In [123]:
an_col = [col for col in transaction_df.columns if (set(transaction_df[col].unique().tolist()) != {0,1})]
duplicate_ind = []
for ind in transaction_df.loc[(transaction_df==2).any(axis=1),an_col].index:
    cond = np.logical_and(np.logical_and(transaction_ori_df['time_id']==ind[0], transaction_ori_df['customer_id']==ind[1]), transaction_ori_df['store_id']==ind[2])
    tmp = transaction_ori_df.loc[cond , 'product_id'].duplicated()
    duplicate_ind.append(tmp.loc[tmp==1].index.values[0])
    
duplicate_ind

[40558,
 41077,
 56547,
 74245,
 10986,
 43894,
 75705,
 35470,
 146929,
 118179,
 45966,
 125466,
 20492,
 47093,
 59781,
 21256,
 46514,
 79656,
 37111,
 22443,
 126118,
 80105,
 150058,
 110529,
 22943,
 22653,
 50889,
 82142]

time: 326 ms (started: 2022-01-04 11:30:01 +00:00)


In [48]:
freq_itemset = apriori(transaction_df, min_support=0.0001, use_colnames=True)
rules = association_rules(freq_itemset, metric='confidence', min_threshold=0.9)
display(freq_itemset.sort_values(by = 'support'))
display(rules)

ValueError: The allowed values for a DataFrame are True, False, 0, 1. Found value 2

time: 378 ms (started: 2022-01-04 06:50:02 +00:00)


In [ ]:
transaction1_tuple_sereis = sales_fact_1998_df[:50].groupby(['time_id','customer_id'])['product_id'].apply(lambda x:tuple(x.astype(str)))
# transaction2 = sales_fact_dec_1998_df.groupby(['time_id','customer_id'])['product_id'].apply(lambda x:set(x))
tmp_1 = pd.DataFrame(transaction1_tuple_sereis.tolist(), index=transaction1_tuple_sereis.index)
tmp_1 = tmp_1.fillna("null")
display(np.unique(tmp_1.values), np.unique(tmp_1.values).shape)

In [ ]:
sales_fact_1998_df[:50].groupby(['time_id','customer_id'])['product_id'].unique()

In [ ]:
dir(sales_fact_1998_df[:50].groupby(['time_id','customer_id'])['product_id'])

In [82]:
tmp_df = pd.DataFrame(np.arange(12).reshape(3,4))
(tmp_df>5).any(axis=1).index

RangeIndex(start=0, stop=3, step=1)

time: 2.99 ms (started: 2022-01-04 07:22:42 +00:00)


In [ ]:
oh_enc = OneHotEncoder()
X = [['Male', 1], ['Female', 3], ['Female', 2]]
oh_tmp = oh_enc.fit(X)
oh_enc.categories_